In [4]:
# https://github.com/ranaroussi/yfinance/wiki
import yfinance as yf
import pandas as pd
import numpy as np
import os

In [5]:
# open sp500.txt file
with open('sp500.txt', 'r') as f:
    symbols = f.read().splitlines()

In [ ]:
count = 1
start = '2000-01-01'
end = '2024-06-16'
df = pd.DataFrame()
for symbol in symbols:
    df = yf.download(symbol, start=start, end=end)
    df.drop_duplicates(inplace=True)

    df.reset_index(inplace=True) # yahoo finance data has date as index, after reset_index, Date is a column
    df.columns = df.columns.str.lower() # make 'Date' to 'date'
    df = df.sort_values(by='date')
    df.reset_index(inplace=True, drop=True) # if rows was swapped, reset index


    df.rename(columns={'adj close':'adj_close'}, inplace=True)
    df = df[['date', 'open', 'high', 'low', 'close', 'volume', 'adj_close']]

    adj_open = df['open'] * df['adj_close'] / df['close']
    df.insert(6, 'adj_open', adj_open)
    adj_high = df['high'] * df['adj_close'] / df['close']
    df.insert(7, 'adj_high', adj_high)
    adj_low = df['low'] * df['adj_close'] / df['close']
    df.insert(8, 'adj_low', adj_low)

    #save 1st table to pickle and excel
    df_cp = df.copy()
    df_cp.to_pickle(f'{symbol}_from_{start}_yfinance.pickle')
    df_cp.loc[:,'date'] = df_cp['date'].astype(str)
    df_cp.to_excel(f'{symbol}_from_{start}_yfinance.xlsx')


    keep_colunms = ['date', 'adj_open', 'adj_high', 'adj_low', 'adj_close', 'volume']
    df = df[keep_colunms]
    df.columns = ['date', 'open', 'high', 'low', 'close', 'volume']

    df['pct_change'] = df['close'].pct_change()
    df['252d_std'] = df['pct_change'].shift(1).rolling(window=252).std(ddof=0)
    df['30d_std'] = df['pct_change'].shift(1).rolling(window=30).std(ddof=0)

    df.to_pickle(f'{symbol}_from_{start}_yfinance_fixed_price.pickle')
    df.loc[:,'date'] = df['date'].astype(str)
    df.to_excel(f'{symbol}_from_{start}_yfinance_fixed_price.xlsx')
    print(f'stock no.{count} - {symbol} downloaded')
    count += 1